# Spark Basics 2

### Combining operations

The method `map` takes as input an RDD and returns an RDD. Similarly the method `reduce` takes as input an RDD and returns a single element. 

We can combine map and reduce operations to perform more complex operations.

Suppose we want to compute the sum of the squares
$$ \sum_{i=1}^n x_i^2 $$
where the elements $x_i$ are stored in an RDD.

Traditional syntax: 
* perform the map
* store the intermediate result in a variable
* perform the reduce

In [ ]:
import sys
import os
import pickle
import numpy as np

testPath = '/'.join(os.getcwd().split('/')[:-1]) + "/Tester"
sys.path.insert(0, testPath )
pickleFile = testPath+ "/SparkBasics2.pkl"

import SparkBasics2

In [ ]:
from pyspark import SparkContext
sc = SparkContext(master="local[4]")
sc

In [ ]:
B=sc.parallelize(range(4))

In [ ]:
#separate commands
Squares=B.map(lambda x:x*x)
Squares.reduce(lambda x,y:x+y)

Or we can combine them into a single cascaded command

In [ ]:
#cascaded commands
B.map(lambda x:x*x)\
   .reduce(lambda x,y:x+y)

These two expressions are equivalent, and we might expect that the more basic one is the first, where the commands 
are separate, and that the python compiler translates the cascaded commands into machine code that corresponds to the separate commands.

It turns out that the opposite is true, it is the cascaded form that is closer to the machine code, and spark identifies cascading operations even when they are expressed in a non-cascaded way.

The explanation of this surprising behaviour is related to the notion of lazy evaluation in scala and is explained in [spark programming guide/RDD operations](http://spark.apache.org/docs/latest/programming-guide.html#rdd-operations)

### An instructive mistake
Here is another way to compute the sum of the squares using a single reduce command. What is wrong with it?

In [ ]:
C=sc.parallelize([1,1,1])
C.reduce(lambda x,y: x*x+y*y)


#### Exercise 1.
Consider the following RDD
    
    listRDD=sc.parallelize([[3,4],[2,1],[7,9]]) 
    
Find the sum of maximum numbers of all lists. Your output should be:

    Output:  15 

# Exercise1: 

In [ ]:
def maxSum(A):
    # Write code that will perform the task outlined in exercise 1
    return "returns the sum of maximum numbers of all lists"

In [ ]:
SparkBasics2.exercise3_1(pickleFile, maxSum,sc)

### getting information about an RDD
RDD's typically have hundreds of thousands of elements. It usually makes no sense to print out the content of a whole RDD. Here are some ways to get manageable amounts of information about an RDD

In [ ]:
n=1000000
B=sc.parallelize([0,0,1,0]*(n/4))

In [ ]:
#find the number of elements in the RDD
B.count()

In [ ]:
# get the first few elements of an RDD
print 'first element=',B.first()
print 'first 5 elements = ',B.take(5)

#### Sampling an RDD
* RDDs are often very large.
* Aggregates, such as averages, can be approximated efficiently by using a sample.
* Sampling is done in parallel and it keeps the data local.

The method `RDD.sample(withReplacement,p)` generates a sample of the elements of the RDD. where
- `withReplacement` is a boolean flag indicating whether or not a an element in the RDD can be sampled more than once.
- `p` is the probability of accepting each element into the sample. Note that as the sampling is performed independently in each partition, the number of elements in the sample changes from sample to sample.

In [ ]:
# get a sample whose expected size is m
m=5.
B.sample(False,m/n).collect()

In [ ]:
#compute the average exactly:
exact=B.reduce(lambda x,y:x+y)/(n+0.0)
print 'exact average', exact
#compute the average by sampling 1% of the elements
p=0.01
approx=B.sample(False,p).reduce(lambda x,y:x+y)/(n*p)
print 'approximate average',approx
print 'error=',approx-exact


#### Things to note and think about
* Each time you run the previous cell, you get a different estimate
* The accuracy of the estimate is determined by the size of the sample $n*p$
* See how the error changes as you vary $p$
* Can you give a formula that relates the variance of the estimate to $(p*n)$ ? (The answer is in the Probability and statistics course).

#### filtering an RDD
The method `RDD.filter(func)` Return a new dataset formed by selecting those elements of the source on which func returns true.


In [ ]:
# How many positive numbers?
B.filter(lambda n: n > 0).count()

#### Exercise 2

Write a `filter` command to output elements whose cosine is positive. On the input 

    RDD=sc.parallelize([0,2,1]):

positiveCos(RDD) should produce some output approximately like:

    PythonRDD[14] at RDD at PythonRDD.scala:48
positiveCos(RDD).collect() should produce:

     [0,1] 
    

In [ ]:
def positiveCos(A):
    # Write code that will perform the task outlined in exercise 2
    return "returns a spark RDD, so to get a list do: positiveCos(A).collect()"

In [ ]:
SparkBasics2.exercise3_2(pickleFile, SparkBasics2_Teacher.func_ex3_2,sc)

#### Exercise 3
Write a `filter` command to output all words whose length is greater than or equal to 4. On the input: ` wordRDD=sc.parallelize(['this','is','the','best','mac','ever']) `

longWords(RDD) should produce some output approximately like:

    PythonRDD[14] at RDD at PythonRDD.scala:48
longWords(RDD).collect() should produce:

    ['this', 'best', 'ever']

In [ ]:
def longWords(A):
    # Write code that will perform the task outlined in exercise 3
    return "returns a spark RDD, so to get a list do: longWords(A).collect()"

In [ ]:
SparkBasics2.exercise3_3(pickleFile, longWords,sc)

#### Remove duplicate elements in RDD
The method `RDD.distinct(numPartitions=None)` Returns a new dataset that contains the distinct elements of the source dataset 

* The number of partitions is specified through the **numPartitions** argument. Each of this partitions is potentially on different machine.


In [ ]:
# Remove duplicate element in DuplicateRDD, we get distinct RDD
DuplicateRDD = sc.parallelize([1,1,2,2,3,3])
DistinctRDD = DuplicateRDD.distinct()
DistinctRDD.collect()


#### flatmap an RDD
The method `RDD.flatMap(func)` is similar to map, but each input item can be mapped to 0 or more output items (so func should return a Seq rather than a single item).

In [ ]:
text=["you are my sunshine","my only sunshine"]
text_file = sc.parallelize(text)
# map each line in text to a list of words
print 'map:',text_file.map(lambda line: line.split(" ")).collect()
# create a single list of words by combining the words from all of the lines
print 'flatmap:',text_file.flatMap(lambda line: line.split(" ")).collect()

#### Exercise 4

Write a `flatMap` command to collect all the elements from 1 to x for each element x in a list. On input `RDD=sc.parallelize([2,3,5])`

collectRange(RDD) should produce some output approximately like:

    PythonRDD[14] at RDD at PythonRDD.scala:48
collectRange(RDD).collect() should produce:

     [1, 2, 1, 2, 3, 1, 2, 3, 4, 5] 

In [ ]:
def collectRange(A):
    # Write code that will perform the task outlined in exercise 4
    return "returns a spark RDD, so to get a list do: collectRange(A).collect()"

In [ ]:
SparkBasics2.exercise3_4(pickleFile, collectRange,sc)

### Set operations
In this part, we explore set operations including **union**,**intersection**,**subtract**, **cartesian** in pyspark

In [ ]:
rdd1 = sc.parallelize([1, 1, 2, 3])
rdd2 = sc.parallelize([1, 3, 4, 5])


1. union(other)
 * Return the union of this RDD and another one.
 

 rdd1.union(rdd2).collect()

2. intersection(other)
 * Return the intersection of this RDD and another one. The output will not contain any duplicate elements, even if the input RDDs did.Note that this method performs a shuffle internally.

In [ ]:
rdd1.intersection(rdd2).collect()

3. subtract(other, numPartitions=None)
 * Return each value in self that is not contained in other.

In [ ]:
rdd1.subtract(rdd2).collect()

4. cartesian(other)
 * Return the Cartesian product of this RDD and another one, that is, the RDD of all pairs of elements (a, b) where **a** is in **self** and **b** is in **other**.

In [ ]:
print rdd1.cartesian(rdd2).collect()

#### Exercise 

Consider the following RDDs: 

` RDD1=sc.parallelize(["spark basics", "big data analysis", "spring"]) `

` RDD2=sc.parallelize(["spark using pyspark", "big data"]) `

Use the set operations for the following exercises:

#### Exercise 5

transform1(RDD1,RDD2) should produce some output approximately like:

    PythonRDD[14] at RDD at PythonRDD.scala:48

transform1(RDD1,RDD2).collect() should produce:

* ` ['spark', 'basics', 'big', 'data', 'analysis', 'spring', 'spark', 'using', 'pyspark', 'big', 'data'] `

In [ ]:
def transform1(RDD1,RDD2):
    # Write code that will perform the task outlined in exercise 5
    return "returns a spark RDD, so to get a list do: transform1(RDD1,RDD2).collect()"

In [ ]:
SparkBasics2.exercise3_5(pickleFile, transform1,sc)

#### Exercise 6

transform2(RDD1,RDD2) should produce some output approximately like:

    PythonRDD[14] at RDD at PythonRDD.scala:48

transform2(RDD1,RDD2).collect() should produce:

* ` ['data', 'big', 'spark'] `

In [ ]:
def transform2(RDD1,RDD2):
    # Write code that will perform the task outlined in exercise 6
    return "returns a spark RDD, so to get a list do: transform2(RDD1,RDD2).collect()"

In [ ]:
SparkBasics2.exercise3_6(pickleFile, transform2,sc)

#### Exercise 7

transform3(RDD1,RDD2) should produce some output approximately like:

    PythonRDD[14] at RDD at PythonRDD.scala:48
transform3(RDD1,RDD2).collect() should produce:

* ` ['spring', 'analysis', 'basics'] `

In [ ]:
def transform3(RDD1,RDD2):
    # Write code that will perform the task outlined in exercise 7
    return "returns a spark RDD, so to get a list do: transform3(RDD1,RDD2).collect()"

In [ ]:
SparkBasics2.exercise3_7(pickleFile, transform3,sc)

#### Exercise 8

transform4(RDD1,RDD2) should produce some output approximately like:

    PythonRDD[14] at RDD at PythonRDD.scala:48
transform4(RDD1,RDD2).collect() should produce:

* ` [('spark', 'spark'), ('spark', 'using'), ('spark', 'pyspark'), ('basics', 'spark'), ('basics', 'using'), ('basics', 'pyspark'), ('spark', 'big'), ('spark', 'data'), ('basics', 'big'), ('basics', 'data'), ('big', 'spark'), ('big', 'using'), ('big', 'pyspark'), ('data', 'spark'), ('analysis', 'spark'), ('data', 'using'), ('data', 'pyspark'), ('analysis', 'using'), ('analysis', 'pyspark'), ('spring', 'spark'), ('spring', 'using'), ('spring', 'pyspark'), ('big', 'big'), ('big', 'data'), ('data', 'big'), ('analysis', 'big'), ('data', 'data'), ('analysis', 'data'), ('spring', 'big'), ('spring', 'data')]
 `    

In [ ]:
def transform4(RDD1,RDD2):
    # Write code that will perform the task outlined in exercise 8
    return "returns a spark RDD, so to get a list do: transform4(RDD1,RDD2).collect()"

In [ ]:
SparkBasics2.exercise3_8(pickleFile, transform4,sc)